# モデルを外れ値のないモデルと組み合わせる


前提として、特徴量エンジニアリングが完了しており、以下の2つのデータセットがあるとします。

- ***train_clean.csv***: トレーニングデータセット
- ***test_clean.csv***: テストデータセット

`train_clean.csv`には1/0の値を持つ**`outlier`**カラムがあります。

さらに、最良のLB提出物があります：
- ***3.695.csv***: （**Ashish Patel(阿希什)**による提出物。オリジナルのモデルではこのスコアに到達できなかったため、このアイデアを使って提出物を改善し、より良いLBスコアを得ることを試みます。）

このパイプラインの流れは以下の通りです：
1. 外れ値のないトレーニングセットを使ってモデルをトレーニングします。（得られるモデルを**`Model_1`**とします）
2. 外れ値を分類するモデルをトレーニングします。（得られるモデルを**`Model_2`**とします）
3. **`Model_2`**を使用してテストセット内のcard_idが外れ値かどうかを予測します。（得られるのは**`Outlier_Likelyhood`**です）
4. **`Outlier_Likelyhood`**のスコアが上位10%（または他の割合）のcard_idを分割します。（得られるのは**`Outlier_ID`**です）
5. 最良の提出物を使って**`Outlier_ID`**をテストセット内で予測し、残りのテストセットは**`Model_1`**を使って予測します。

このパイプラインの基本的なアイデアは以下の通りです：
1. 外れ値のないモデルをトレーニングすることで、非外れ値に対してモデルの精度を向上させる。
2. 多くのエラーは外れ値によって引き起こされるため、外れ値を予測するためのモデルが必要です。それらを見つける方法は？分類器を構築することです！


In [1]:
import numpy as np
import pandas as pd
import time
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss

# Part 1 Training Model Without Outliers

In [2]:
%%time
df_train = pd.read_csv('../input/predicting-outliers-to-improve-your-score/train_clean.csv')
df_test = pd.read_csv('../input/predicting-outliers-to-improve-your-score/test_clean.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../input/predicting-outliers-to-improve-your-score/train_clean.csv'

: 

## filtering out outliers

In [ ]:
df_train = df_train[df_train['outliers'] == 0]
target = df_train['target']
del df_train['target']
features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','outliers']]
categorical_feats = [c for c in features if 'feature_' in c]

## parameters

In [ ]:
param = {'objective':'regression',
         'num_leaves': 31,
         'min_data_in_leaf': 25,
         'max_depth': 7,
         'learning_rate': 0.01,
         'lambda_l1':0.13,
         "boosting": "gbdt",
         "feature_fraction":0.85,
         'bagging_freq':8,
         "bagging_fraction": 0.9 ,
         "metric": 'rmse',
         "verbosity": -1,
         "random_state": 2333}

## training model

In [ ]:
%%time
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2333)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval= 100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))

In [ ]:
model_without_outliers = pd.DataFrame({"card_id":df_test["card_id"].values})
model_without_outliers["target"] = predictions

# Part 2 Training Model For Outliers Classification

In [ ]:
%%time
df_train = pd.read_csv('../input/predicting-outliers-to-improve-your-score/train_clean.csv')
df_test = pd.read_csv('../input/predicting-outliers-to-improve-your-score/test_clean.csv')

## using outliers column as labels instead of target column

In [ ]:
target = df_train['outliers']
del df_train['outliers']
del df_train['target']

In [ ]:
features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month']]
categorical_feats = [c for c in features if 'feature_' in c]

## parameters

In [ ]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 6,
         'learning_rate': 0.01,
         "boosting": "rf",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'binary_logloss',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "random_state": 2333}

## training model

In [ ]:
%%time
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

start = time.time()


for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(log_loss(target, oof)))

In [ ]:
### 'target' is the probability of whether an observation is an outlier
df_outlier_prob = pd.DataFrame({"card_id":df_test["card_id"].values})
df_outlier_prob["target"] = predictions
df_outlier_prob.head()


# パート3：提出物の組み合わせ
ここまで順調です！
今、3つのデータセットがあります：

1. 最良の提出物
2. 外れ値のないモデルを使用した予測
3. テストセット内の外れ値の確率


In [ ]:
# テストセットにトレーニングセットと同じ割合の外れ値がある場合、 
# テストセットの外れ値の数は約次の通りです：（トレーニングセットの外れ値は1.06％）
123623*0.0106

In [ ]:
# 予測可能な外れ値を見逃す場合に備えて、外れ値の可能性が最も高い上位25,000を選択します。
outlier_id = pd.DataFrame(df_outlier_prob.sort_values(by='target',ascending = False).head(25000)['card_id'])

In [ ]:
best_submission = pd.read_csv('../input/predicting-outliers-to-improve-your-score/3.695.csv')

In [ ]:
most_likely_liers = best_submission.merge(outlier_id,how='right')
most_likely_liers.head()

In [ ]:
%%time
for card_id in most_likely_liers['card_id']:
    model_without_outliers.loc[model_without_outliers['card_id']==card_id,'target']\
    = most_likely_liers.loc[most_likely_liers['card_id']==card_id,'target'].values

In [ ]:
model_without_outliers.to_csv("combining_submission.csv", index=False)